<h1>Import modules

In [1]:
import pandas as pd
import numpy as np
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import folium
from geopy.geocoders import Nominatim
from pandas.io.json import json_normalize

<h1>Obtain data from wikipedia page

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data  = requests.get(url).text
soup = BeautifulSoup(data,"html5lib")
tables = soup.find_all('table')

<h1>Create new DataFrame with only three columns

In [3]:
cell = {'PostalCode':'', 'Borough':'', 'Neighbourhood':''}

<h1>Scan table element in the page and assign the respective values to their fields in the DataFrame

In [4]:
table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

<h1>Print the shape of the DataFrame

In [5]:
df.shape

(103, 3)

<h1>Import DataFrame with Geolocation for the PostalCodes

In [6]:
df_data_1 = pd.read_csv('Geospatial_Coordinates.csv')

<h1>Join the Latitude and Longitude columns with the previously created DataFrame "df"

In [7]:
df = df.join(df_data_1.set_index('Postal Code'), on='PostalCode')

In [8]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


<h1>Set the whole set of neighborhoods in Toronto as the dataframe, no filter is done

In [66]:
toronto_data = df.reset_index(drop=True)
toronto_data.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


<h1>Set the city as the address to look for its latitude and longitude

In [22]:
address = 'Toronto'

geolocator = Nominatim(user_agent="kino-bot")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


<h1>Create a map of Toronto with all the neighboorhoods identified

In [23]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, label in zip(toronto_data['Latitude'], toronto_data['Longitude'], toronto_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

<h1>Set the credentials for the Foursquare API (Client data hidden for security reasons)

In [80]:
CLIENT_ID = '***'
CLIENT_SECRET = '***'
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value
ACCESS_TOKEN = '***'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: ***
CLIENT_SECRET:***


In [25]:
toronto_data.loc[0, 'Neighborhood']
neighborhood_latitude = toronto_data.loc[0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[0, 'Neighborhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Parkwoods are 43.7532586, -79.3296565.


<h1>Create an URL for the foursquare API based on the credentials and the neighboorhood lat and long values

In [29]:
search_item = 'Moore Park, Summerhill East'
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&query={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude,  
    5000, 
    search_item, 
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=M3AN4CLICWYHRSEWXX31JZG4EWTINJTLVI1E0UY0MWTLTPY4&client_secret=GTENCABFYVUL1XZQGT2IC3NOWW4DCQIMPQGZDBSX4SKVYHL2&v=20180605&ll=43.7532586,-79.3296565&radius=5000&query=Moore Park, Summerhill East&limit=100'

In [30]:
results = requests.get(url).json()

In [31]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [32]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-32-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Delta Hotels by Marriott Toronto East,Hotel,43.777070,-79.282822
1,Graydon Hall Park,Playground,43.764114,-79.345590
2,Hwy 401 & Victoria Park Ave,Intersection,43.768351,-79.319744
3,Lawrence Ave E & Victoria Park Ave,Intersection,43.742276,-79.310094
4,East Don Trail,Trail,43.734417,-79.328766


In [33]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

9 venues were returned by Foursquare.


In [34]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [35]:
nearby_venues = json_normalize(venues) # flatten JSON
nearby_venues

<ipython-input-35-ea6da929ed3c>:1: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,...,venue.location.postalCode,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.neighborhood
0,e-0-4ad4c05cf964a520c6f520e3-0,0,"[{'summary': 'This spot is popular', 'type': '...",4ad4c05cf964a520c6f520e3,Delta Hotels by Marriott Toronto East,2035 Kennedy Road,at Village Green Sq,43.777070,-79.282822,"[{'label': 'display', 'lat': 43.7770701, 'lng'...",...,M1T 3G2,CA,Toronto,ON,Canada,"[2035 Kennedy Road (at Village Green Sq), Toro...","[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",0,[],NaN
1,e-0-511d6481e4b0b7ae65e15c51-1,0,"[{'summary': 'This spot is popular', 'type': '...",511d6481e4b0b7ae65e15c51,Graydon Hall Park,North York Division,Graydon Hall Dr. & Don Mills Rd.,43.764114,-79.345590,"[{'label': 'display', 'lat': 43.76411391626431...",...,NaN,CA,North York,ON,Canada,[North York Division (Graydon Hall Dr. & Don M...,"[{'id': '4bf58dd8d48988d1e7941735', 'name': 'P...",0,[],NaN
2,e-0-4f0f2aade4b04ebcbb6448be-2,0,"[{'summary': 'This spot is popular', 'type': '...",4f0f2aade4b04ebcbb6448be,Hwy 401 & Victoria Park Ave,Highway 401,at Victoria Park Avenue,43.768351,-79.319744,"[{'label': 'display', 'lat': 43.7683511778946,...",...,NaN,CA,Toronto,ON,Canada,"[Highway 401 (at Victoria Park Avenue), Toront...","[{'id': '52f2ab2ebcbc57f1066b8b4c', 'name': 'I...",0,[],Wexford
3,e-0-4cd9d59e8728721eaa5c1489-3,0,"[{'summary': 'This spot is popular', 'type': '...",4cd9d59e8728721eaa5c1489,Lawrence Ave E & Victoria Park Ave,Lawrence Avenue East,at Victoria Park Avenue,43.742276,-79.310094,"[{'label': 'display', 'lat': 43.74227576539719...",...,NaN,CA,Toronto,ON,Canada,[Lawrence Avenue East (at Victoria Park Avenue...,"[{'id': '52f2ab2ebcbc57f1066b8b4c', 'name': 'I...",0,[],Parkwoods - Donalda
4,e-0-554135b1498e60fc30be40d4-4,0,"[{'summary': 'This spot is popular', 'type': '...",554135b1498e60fc30be40d4,East Don Trail,NaN,NaN,43.734417,-79.328766,"[{'label': 'display', 'lat': 43.734417, 'lng':...",...,M3A 1A1,CA,Toronto,ON,Canada,"[Toronto ON M3A 1A1, Canada]","[{'id': '4bf58dd8d48988d159941735', 'name': 'T...",0,[],NaN
5,e-0-4b47ba6ef964a5202d3c26e3-5,0,"[{'summary': 'This spot is popular', 'type': '...",4b47ba6ef964a5202d3c26e3,East Side Mario's Scarborough,12 Lebovic Ave Unit 9,at Eglinton Ave. E,43.723914,-79.290818,"[{'label': 'display', 'lat': 43.7239137946718,...",...,M1L 4V9,CA,Scarborough,ON,Canada,"[12 Lebovic Ave Unit 9 (at Eglinton Ave. E), S...","[{'id': '4bf58dd8d48988d1c4941735', 'name': 'R...",0,[],Clairlea - Birchmount
6,e-0-4e934cbc93adf0b000acb514-6,0,"[{'summary': 'This spot is popular', 'type': '...",4e934cbc93adf0b000acb514,East Don Parklands Trail,NaN,NaN,43.771511,-79.364208,"[{'label': 'display', 'lat': 43.7715113782369,...",...,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]","[{'id': '4bf58dd8d48988d159941735', 'name': 'T...",0,[],NaN
7,e-0-4c1173bc72caa593148f5da4-7,0,"[{'summary': 'This spot is popular', 'type': '...",4c1173bc72caa593148f5da4,TTC Bus #34 Eglinton East,Eglinton Ave. E.,NaN,43.720802,-79.334556,"[{'label': 'display', 'lat': 43.72080201490044...",...,NaN,CA,Toronto,ON,Canada,"[Eglinton Ave. E., Toronto ON, Canada]","[{'id': '4bf58dd8d48988d12b951735', 'name': 'B...",0,[],NaN
8,e-0-4b0bdcc3f964a520363423e3-8,0,"[{'summary': 'This spot is popular', 'type': '...",4b0bdcc3f964a520363423e3,Lawrence East RT Station,2444 Lawrence Ave E,at Midland Ave.,43.750383,-79.270192,"[{'label': 'display', 'lat': 43.75038338963791...",...,M1P 4S5,CA,Toronto,ON,Canada,"[2444 Lawrence Ave E (at Midland Ave.), Toront...","[{'id': '4bf58dd8d48988d1fc931735', 'name': 'L...",0,[],NaN


In [36]:
toronto_venues = getNearbyVenues(toronto_data['Neighborhood'], toronto_data['Latitude'], toronto_data['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

In [37]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",6,6,6,6,6,6
"Bathurst Manor, Wilson Heights, Downsview North",22,22,22,22,22,22
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",26,26,26,26,26,26
...,...,...,...,...,...,...
"Willowdale, Newtonbrook",1,1,1,1,1,1
Woburn,4,4,4,4,4,4
Woodbine Heights,5,5,5,5,5,5


In [38]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 272 uniques categories.


In [39]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [40]:
toronto_onehot.shape

(2122, 272)

In [41]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"Willowdale, Newtonbrook",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [42]:
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Train Station,Truck Stop,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"Willowdale, Newtonbrook",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
96,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
97,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
98,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [43]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Agincourt----
                       venue  freq
0                     Lounge   0.2
1             Breakfast Spot   0.2
2             Clothing Store   0.2
3               Skating Rink   0.2
4  Latin American Restaurant   0.2


----Alderwood, Long Branch----
            venue  freq
0     Pizza Place  0.33
1  Sandwich Place  0.17
2     Coffee Shop  0.17
3             Pub  0.17
4             Gym  0.17


----Bathurst Manor, Wilson Heights, Downsview North----
         venue  freq
0  Coffee Shop  0.09
1         Bank  0.09
2    Pet Store  0.05
3  Gas Station  0.05
4         Park  0.05


----Bayview Village----
                 venue  freq
0                 Café  0.25
1  Japanese Restaurant  0.25
2                 Bank  0.25
3   Chinese Restaurant  0.25
4          Yoga Studio  0.00


----Bedford Park, Lawrence Manor East----
                venue  freq
0  Italian Restaurant  0.12
1         Coffee Shop  0.08
2          Restaurant  0.08
3      Sandwich Place  0.08
4   Indian Restaurant  0.04

                venue  freq
0    Business Service  0.25
1                Park  0.25
2         Swim School  0.25
3            Bus Line  0.25
4  Miscellaneous Shop  0.00


----Leaside----
                    venue  freq
0             Coffee Shop  0.09
1     Sporting Goods Shop  0.09
2            Burger Joint  0.06
3  Furniture / Home Store  0.06
4                    Bank  0.06


----Little Portugal, Trinity----
                           venue  freq
0                            Bar  0.11
1                    Coffee Shop  0.07
2  Vegetarian / Vegan Restaurant  0.05
3                    Men's Store  0.05
4               Asian Restaurant  0.05


----Malvern, Rouge----
                             venue  freq
0                       Print Shop   0.5
1             Fast Food Restaurant   0.5
2                    Metro Station   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0


----Milliken, Agincourt North, Steeles East, L'Amoreaux East----
               

                             venue  freq
0                      Coffee Shop  0.50
1                     Soccer Field  0.25
2            Korean BBQ Restaurant  0.25
3              Monument / Landmark  0.00
4  Molecular Gastronomy Restaurant  0.00


----Woodbine Heights----
          venue  freq
0  Intersection   0.2
1          Park   0.2
2    Beer Store   0.2
3  Skating Rink   0.2
4   Curling Ice   0.2


----York Mills West----
                             venue  freq
0                             Park  0.67
1                Convenience Store  0.33
2               Mexican Restaurant  0.00
3              Monument / Landmark  0.00
4  Molecular Gastronomy Restaurant  0.00


----York Mills, Silver Hills----
                             venue  freq
0                        Cafeteria   1.0
1        Middle Eastern Restaurant   0.0
2              Moroccan Restaurant   0.0
3              Monument / Landmark   0.0
4  Molecular Gastronomy Restaurant   0.0




In [44]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [45]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Lounge,Breakfast Spot,Clothing Store,Skating Rink,Latin American Restaurant,Yoga Studio,Miscellaneous Shop,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant
1,"Alderwood, Long Branch",Pizza Place,Sandwich Place,Coffee Shop,Pub,Gym,Health Food Store,Medical Center,Men's Store,Metro Station,Mexican Restaurant
2,"Bathurst Manor, Wilson Heights, Downsview North",Coffee Shop,Bank,Pet Store,Gas Station,Park,Shopping Mall,Middle Eastern Restaurant,Mobile Phone Shop,Sandwich Place,Supermarket
3,Bayview Village,Café,Japanese Restaurant,Bank,Chinese Restaurant,Yoga Studio,Modern European Restaurant,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Mobile Phone Shop
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Coffee Shop,Restaurant,Sandwich Place,Indian Restaurant,Juice Bar,Sushi Restaurant,Café,Liquor Store,Butcher


In [46]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 0], dtype=int32)

In [47]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [73]:
toronto_merged = toronto_data

# merge manhattan_grouped with manhattan_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

<h1>Drop all NaN rows

In [74]:
toronto_merged = toronto_merged.dropna() 

<h1>Create a map with the neighborhoods in clusters based on their similarity

In [77]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

<h1>The map above suggests that most of the neighborhoods fall into the same cluster 1.0, containing mostly Coffee Shops</h1>

In [68]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,value_is_NaN
0,North York,0.0,Park,Food & Drink Shop,Yoga Studio,Mexican Restaurant,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Metro Station,No
14,East York,0.0,Intersection,Park,Beer Store,Skating Rink,Curling Ice,Miscellaneous Shop,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,No
21,York,0.0,Park,Women's Store,Pool,Mexican Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,No
35,East York/East Toronto,0.0,Pizza Place,Park,Metro Station,Convenience Store,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Martial Arts School,Motel,No
40,North York,0.0,Bus Stop,Airport,Park,Other Repair Shop,Yoga Studio,Middle Eastern Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,No
46,North York,0.0,Shopping Mall,Grocery Store,Bank,Park,Yoga Studio,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,No
49,North York,0.0,Bakery,Park,Construction & Landscaping,Basketball Court,Yoga Studio,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,No
52,North York,0.0,Park,Yoga Studio,Mexican Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Metro Station,No
61,Central Toronto,0.0,Business Service,Park,Swim School,Bus Line,Miscellaneous Shop,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Yoga Studio,No
64,York,0.0,Park,Convenience Store,Mexican Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Yoga Studio,No


In [69]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,value_is_NaN
1,North York,1.0,Intersection,Pizza Place,Coffee Shop,Portuguese Restaurant,Hockey Arena,Miscellaneous Shop,Motel,Moroccan Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,No
2,Downtown Toronto,1.0,Coffee Shop,Park,Bakery,Theater,Pub,Restaurant,Café,Breakfast Spot,Yoga Studio,Distribution Center,No
3,North York,1.0,Clothing Store,Furniture / Home Store,Women's Store,Coffee Shop,Miscellaneous Shop,Carpet Store,Boutique,Accessories Store,Vietnamese Restaurant,Other Repair Shop,No
4,Queen's Park,1.0,Coffee Shop,Sushi Restaurant,Yoga Studio,Italian Restaurant,Burrito Place,Café,College Auditorium,College Cafeteria,Creperie,Diner,No
6,Scarborough,1.0,Print Shop,Fast Food Restaurant,Metro Station,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,Yoga Studio,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,Downtown Toronto,1.0,Coffee Shop,Restaurant,Bakery,Pizza Place,Italian Restaurant,Pub,Café,Caribbean Restaurant,Beer Store,Liquor Store,No
97,Downtown Toronto,1.0,Coffee Shop,Café,Hotel,Gym,Restaurant,Japanese Restaurant,Salad Place,Seafood Restaurant,Deli / Bodega,Steakhouse,No
99,Downtown Toronto,1.0,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Mediterranean Restaurant,Men's Store,Fast Food Restaurant,Pub,Hotel,No
100,East Toronto Business,1.0,Light Rail Station,Fast Food Restaurant,Garden Center,Pizza Place,Recording Studio,Restaurant,Burrito Place,Skate Park,Brewery,Smoke Shop,No


In [70]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,value_is_NaN
53,North York,2.0,Food Truck,Baseball Field,Yoga Studio,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Mexican Restaurant,Moroccan Restaurant,No
57,North York,2.0,Baseball Field,Yoga Studio,Middle Eastern Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Mexican Restaurant,Luggage Store,No
101,Etobicoke,2.0,Baseball Field,Yoga Studio,Middle Eastern Restaurant,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Mexican Restaurant,Luggage Store,No


In [71]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,value_is_NaN
51,Scarborough,4.0,American Restaurant,Motel,Yoga Studio,Monument / Landmark,Molecular Gastronomy Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant,Mexican Restaurant,No
